# Language Modeling using Ngram

In this Exercise, you are going to use NLTK which is a natural language processing library for python to create a bigram language model and its variation. You will build one model for each of the following type and calculate their perplexity:
- Unigram Model
- Bigram Model
- Bigram Model with Laplace smoothing
- Bigram Model with Interpolation
- Bigram Model with Kneser-ney Interpolation
- Neural LM (optional)



In [ ]:
#download corpus
!wget --no-check-certificate https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
!unzip BEST2010.zip

--2021-02-13 04:55:34--  https://github.com/ekapolc/nlp_2019/raw/master/HW4/BEST2010.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip [following]
--2021-02-13 04:55:34--  https://raw.githubusercontent.com/ekapolc/nlp_2019/master/HW4/BEST2010.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7423530 (7.1M) [application/zip]
Saving to: ‘BEST2010.zip.1’

BEST2010.zip.1      100%[===================>]   7.08M  41.2MB/s    in 0.2s    

2021-02-13 04:55:35 (41.2 MB/s) - ‘BEST2010.zip.1’ saved [7423530/7423530]

Archive:  BEST2010.zip
replace BEST2010/a

In [ ]:
#First we import necessary library such as math, nltk, bigram, and collections.
import math
import nltk
import io
import random
from random import shuffle
from nltk import bigrams, trigrams
from collections import Counter, defaultdict
random.seed(999)

BEST2010 is a free Thai NLP dataset by NECTEC usually use as a standard benchmark for various NLP tasks includeing language modeling. BEST2010 is separated into 4 domain article, encyclopedia, news and novel. The data is already  tokenized using '|' as a separator.

For example,

ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น|

In [ ]:
# We choose news domain as our dataset
best2010=[]
fp= io.open('BEST2010/news.txt','r',encoding='utf-8')
for i,line in enumerate(fp):
    best2010.append(line.strip()[:-1])
fp.close()
all_vocabulary =set()
total_word_count =0
for line in best2010:
    for word in line.split('|'):        
        all_vocabulary.add(word)
        total_word_count+=1

In [ ]:
#For simplicity, we assumes that each line is a sentence.
print ('Total sentences in BEST2010 news dataset :\t'+ str(len(best2010)))
print ('Total word counts in BEST2010 news dataset :\t'+ str(total_word_count))
print ('Total vocabulary in BEST2010 news dataset :\t'+ str(len(all_vocabulary)))

Total sentences in BEST2010 news dataset :	30969
Total word counts in BEST2010 news dataset :	1660190
Total vocabulary in BEST2010 news dataset :	35488


We separate out input into 2 sets, train and test data with 70:30 ratio

In [ ]:
sentences = best2010
# The data is separated to train and test set with 70:30 ratio.
train = sentences[:int(len(sentences)*0.7)]
test = sentences[int(len(sentences)*0.7):]

#Training data
train_vocabulary =set()
train_word_count =0
for line in train:
    for word in line.split('|'):        
        train_vocabulary.add(word)
        train_word_count+=1
print ('Total sentences in BEST2010 news training dataset :\t'+ str(len(train)))
print ('Total word counts in BEST2010 news training dataset :\t'+ str(train_word_count))
print ('Total vocabuary in BEST2010 news training dataset :\t'+ str(len(train_vocabulary)))
# We will use 1/vocab_size as a default value for unknown word
unk_value = math.pow(len(train_vocabulary),-1)

Total sentences in BEST2010 news training dataset :	21678
Total word counts in BEST2010 news training dataset :	1042797
Total vocabuary in BEST2010 news training dataset :	26240


# Unigram

In this section, we will demonstrate how to build a unigram language model <br>
**Important note:** <br>
**\<s\>** = sentence start symbol <br>
**\</s\>** = sentence end symbol 

In [ ]:
def getUnigramModel(data):
    model = defaultdict(lambda: 0)
    word_count = 0
    for sentence in data:
        sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
        for w1 in sentence.split('|'):
            model[w1] +=1.0
            word_count+=1
    for w1 in model:
        model[w1] = model[w1]/(word_count)
    return model

In [ ]:
model = getUnigramModel(train)

In [ ]:
def getLnValue(x):
    if x >0.0:
        return math.log(x)
    else:
        return math.log(unk_value)

In [ ]:
#problability of 'นายก'
print(getLnValue(model[u'นายก']))
#for example, problability of 'นายกรัฐมนตรี' which is an unknown word is equal to
print(getLnValue(model[u'นายกรัฐมนตรี']))
#problability of 'นายก' 'ได้' 'ให้' 'สัมภาษณ์' 'กับ' 'สื่อ'
prob = getLnValue(model[u'นายก'])+getLnValue(model[u'ได้'])+ getLnValue(model[u'ให้'])+getLnValue(model[u'สัมภาษณ์'])+getLnValue(model[u'กับ'])+getLnValue(model[u'สื่อ'])+getLnValue(model['</s>'])
print ('Problability of a sentence', math.exp(prob))

-6.551526663995246
-10.175040243058024
Problability of a sentence 5.617210748667918e-18


# Perplexity

In order to compare language model we need to calculate perplexity. In this task you should write a perplexity calculation code for the unigram model. The result perplexity should be around 556.39 and
476.07 on train and test data.

# TODO #0
**Fill your name and ID here** <br>
**Name**: Wisaroot Lertthaweedech
<br>
**ID**: 6370267221

## TODO #1 **Calculate perplexity**

In [ ]:
# train[:5]

In [ ]:
# model

In [ ]:
def calculate_sentence_ln_prob(sentence, model):
  sum_ln_prop = 0
  word_count = 0
  for w1 in sentence.split('|'):
    sum_ln_prop += getLnValue(model[w1])
    word_count += 1

  return sum_ln_prop, word_count

def perplexity(test,model):
  sum_ln_prop = 0
  word_count = 0
  for sentence in test:
    sentence +=  u'|</s>'
    s, w = calculate_sentence_ln_prob(sentence, model)
    sum_ln_prop += s
    word_count += w

  sum_ln_prop = -1*sum_ln_prop/word_count
  sum_ln_prop = math.exp(sum_ln_prop)
  
  return sum_ln_prop

In [ ]:
print(perplexity(train,model))
print(perplexity(test,model))

556.3925994212195
476.0687892303532


## TODO #2 **Please explain why this model give us such a high perplexity.**

**Your answer**:  

The model predict the next word only based on the frequency of the word itself. Not adjusted the probability based on the previous word at all. So, there will be the low frequency word in train set that the model need to predict so many time and pull perplexity, the average time of guessing for each word, to be very high.

# Bigram

Next, you will create a better language model than a unigram (which is not much to compare with). But first, it is very tedious to count every pair of words that occur in our corpus by ourselves. In this case, nltk provide us a simple library which will do it for us.

In [ ]:
#example of nltk usage for bigram
sentence = 'I always search google for an answer .'

print('This is how nltk generate bigram.')
for w1,w2 in bigrams(sentence.split(), pad_right=True, pad_left=True):
    print (w1,w2)
print('None is used as a start and end of sentence symbol.')

This is how nltk generate bigram.
None I
I always
always search
search google
google for
for an
an answer
answer .
. None
None is used as a start and end of sentence symbol.


Now, you should be able to implement a bigram model by yourself. Also, you must create a new perplexity calculation for bigram. The result perplexity should be around 58.78 and 146.26 on train and test data.

## TODO #3 **Create a Bigram Model**

In [ ]:
# train

In [ ]:
def getBigramModel(data):
    ###FILL YOUR CODE HERE###
    model = defaultdict(lambda: 0)
    unigram_count = defaultdict(lambda: 0)
    word_count = 0
    for sentence in data:
        for (w1, w2) in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          model[w1, w2] += 1
          unigram_count[w1] += 1

    for k,v in model.items(): 
      model[k[0], k[1]] = model[k[0], k[1]]/unigram_count[k[0]]
    return model

model = getBigramModel(train)

In [ ]:
# model['อย.', 'ระบุ']

In [ ]:
model[None, 'สงสัย']

0.00018451886705415628

In [ ]:
# model

## TODO #4 **Calculate Perplexity for Bigram Model**



In [ ]:
test[6]

'10| |คน| |ขณะ|นี้|ไม่|มี|คน|ไข้|เข้า|มา|เลย'

In [ ]:
calculate_sentence_ln_prob('ไข้', model)

(-10.175040243058024, 1)

In [ ]:
calculate_sentence_ln_prob(test[6], model)

(-132.2755231597543, 13)

In [ ]:
def calculate_sentence_ln_prob(sentence, model):
  sum_ln_prop = 0
  word_count_s = 0
  sentence = sentence.split('|')
  for i in range(len(sentence)): 
    if len(sentence) == 1:
      sum_ln_prop += getLnValue(model[None, sentence[i]])
      sum_ln_prop += getLnValue(model[sentence[i], None])
      word_count_s+=2
      continue
    if i == 0:
      sum_ln_prop += getLnValue(model[None, sentence[i]])
      word_count_s+=1
    elif i == len(sentence)-1:
      sum_ln_prop += getLnValue(model[sentence[i-1], sentence[i]])
      sum_ln_prop += getLnValue(model[sentence[i], None])
      word_count_s+=2
    else:
      sum_ln_prop += getLnValue(model[sentence[i-1], sentence[i]])
      word_count_s+=1
  
  return sum_ln_prop, word_count_s

def perplexity(test,model):
  sum_ln_prop = 0
  word_count = 0
  for sentence in test:
    s, w = calculate_sentence_ln_prob(sentence, model)
    sum_ln_prop += s
    word_count += w
  
  sum_ln_prop = -1*sum_ln_prop/(word_count)
  sum_ln_prop = math.exp(sum_ln_prop)

  return sum_ln_prop

In [ ]:
print (perplexity(train,model))
print (perplexity(test, model))

# 58.78942889767147
# 146.26539331038614

58.78942889767147
146.26539331038614


# Smoothing

Usually any ngram models have a sparsity problem, which means it does not have every possible ngram of words in the dataset. Smoothing techniques can alleviate this problem. In this section, you will implement two basic smoothing methods laplace smoothing and interpolation for bigram.

## TODO #5 **Bigram with Laplace smoothing (Add-One Smoothing)**

In [ ]:
train[:5]

['สงสัย|ติด|หวัด|นก| |อีก|คน|ยัง|น่า|ห่วง',
 'ตาม|ที่|นางประนอม ทองจันทร์| |กับ| |ด.ช.กิตติพงษ์ แหลมผักแว่น| |และ| |ด.ญ.กาญจนา กรองแก้ว| |ป่วย|สงสัย|ติด|เชื้อ|ไข้|ขณะ|นี้|ยัง|ไม่|ดี|ขึ้น',
 'หลัง|เข้า|เยี่ยม|ดู|อาการ|ผู้|ป่วย|แล้ว| |น.พ.จรัล|ประชุม|ร่วม|กับ|เจ้าหน้าที่|ทุก|ฝ่าย| |เพื่อ|สรุป|ผล|การ|ดำเนิน|การ| |รวม|ทั้ง|สอบสวน|โรค|ก่อน|ที่|ผู้|ป่วย|จะ|ถูก|ส่ง|มา|รักษา|ตัว| |จาก|นั้น|ร่วม|กัน|แถลง|ข่าว| |โดย| |น.พ.จรัล|กล่าว|ว่า| |ขณะ|นี้|ผู้|ป่วย|ทั้ง| |3| |ราย| |อาการ|ยัง|ทรง| |โดย|ใน|ราย|ของ| |ด.ช.กิตติพงษ์| |กับ| |ด.ญ.กาญจนา| |ปอด|หาย|เป็น|ปกติ|แล้ว| |คาด|ว่า|จะ|กลับ|บ้าน|ได้|ใน|ไม่|ช้า|นี้| |แต่|ใน|ราย|ของ|นางประนอม|อาการ|ยัง|น่า|เป็นห่วง| |ซึ่ง|ทั้ง| |3| |ราย| |ใน|ชั้น|นี้|ถือ|ว่า|เป็น|ผู้|ป่วย|อยู่|ใน|ขั้น|น่า|สงสัย|อาจ|ติด|เชื้อ|ไข้|หวัด|นก| |เพราะ|ตรวจ|พบ|ผู้|ป่วย|มี|อาการ|ปอด|บวม|ปอด|อักเสบ| |เนื่อง|จาก|ติด|เชื้อ|ไวรัส| |แต่|ยัง|สรุป|ไม่|ได้|ว่า|ติด|เชื้อ|ไข้|หวัด|นก|แน่ชัด|หรือ|ไม่| |ต้อง|รอ|ผล|ตรวจ|จาก|ห้อง|ปฏิบัติการ|ที่|ได้|ส่ง|ตัวอย่าง|เลือด| |ไป|ตรวจ|พิสูจน์|ที่|กรมวิทยาศ

In [ ]:
#Laplace Smoothing
def getBigramWithLaplaceSmoothing(data):
    #Fill code here
    model = defaultdict(lambda: 0)
    unigram_count = defaultdict(lambda: 0)
    word_count = 0
    for sentence in data:
        for (w1, w2) in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          model[w1, w2] += 1
          unigram_count[w1] += 1

    for k,v in model.items(): 
      model[k[0], k[1]] = (model[k[0], k[1]]+1)/(unigram_count[k[0]]+len(unigram_count))
    return model

model = getBigramWithLaplaceSmoothing(train)
print (perplexity(train,model) )
print (perplexity(test, model))

# 974.8134581679766
# 1098.1622194979489

974.8134581679766
1098.1622194979489


## TODO #6 **Bigram with Interpolation**
lambda value is 0.7 for bigram, 0.25 for unigram, and 0.05 for unknown word

In [ ]:
# def getUnigramModel(data):
#     model = defaultdict(lambda: 0)
#     word_count = 0
#     for sentence in data:
#         sentence +=  u'|</s>' #for unigram model we can always ignore <s>, since p(w0=<s>)=1
#         for w1 in sentence.split('|'):
#             model[w1] +=1.0
#             word_count+=1
#     for w1 in model:
#         model[w1] = model[w1]/(word_count)
#     return model

In [ ]:
# test[1]

In [ ]:
# for (w1, w2) in bigrams(test[1].split('|'), pad_right=True, pad_left=True):
#   print(str(w1) + ' + ' + str(w2))

In [ ]:
#interpolation
def getBigramWithInterpolation(data):
    #Fill code here
    bigram_count = defaultdict(lambda: 0)
    unigram_count = defaultdict(lambda: 0)
    model = defaultdict(lambda: 0)
    word_count = 0

    for sentence in data:
      for (w1, w2) in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
        bigram_count[w1, w2] += 1
        unigram_count[w2] += 1
        word_count += 1
      word_count -= 1

    for k,v in bigram_count.items(): 
      model[k[0], k[1]] = 0.7*(bigram_count[k[0], k[1]]/unigram_count[k[0]]) + \
                          0.25*(unigram_count[k[1]]/word_count) + \
                          0.05*(1/(len(unigram_count)))
    return model
    
model = getBigramWithInterpolation(train)
print (perplexity(train,model))        
print (perplexity(test,model))

# 73.38409869825665
# 172.67485908813356

73.38410880445593
172.67487186736156


In [ ]:
# model['สงสัย', 'ติด']

the result perplexity on training and testing should be 

    974.81, 1098.16 for Laplace smoothing
    73.38, 172.67 for Interpolation

# Language modeling on multiple domains

Sometimes, we do not have enough data to create a language model for a new domain. In that case, we can improvised by combining several models to improve result on the new domain.

In this exercise you will try to merge two language models from news and article domains to create a language model for the encyclopedia domain.

In [ ]:
# # create article data
# encyclo_data=[]
# fp= io.open('BEST2010/encyclopedia.txt','r',encoding='utf-8')
# for i,line in enumerate(fp):
#     encyclo_data.append(line.strip()[:-1])
# fp.close()

In [ ]:
# We choose news domain as our dataset
def sentences_data(data_set = ['news']):
  best2010=[]
  for i in data_set:
    fp= io.open('BEST2010/'+ i +'.txt','r',encoding='utf-8')
    for i,line in enumerate(fp):
        best2010.append(line.strip()[:-1])
    fp.close()
  return best2010

First, you should try to calculate perplexity of your bigram with interpolation using "news data" (train) on "encyclopedia data" (test). The result perplexity should be around 727.35.

For your information, a bigram model with interpolation using "ariticle data" (train) to test on "encyclopedia data" (test) has a perplexity of 505.79.

In [ ]:
# print perplexity of bigram with interpolation on article data        
# 727.3502637212223
encyclo_data = sentences_data(['encyclopedia'])
print (perplexity(encyclo_data,model))

727.3503259715216


In [ ]:
# article_data = sentences_data(['article'])
# model = getBigramWithInterpolation(article_data)   
# print (perplexity(article_data,model))
# print (perplexity(encyclo_data,model))

## TODO #7 
Write a model that produce 450.0 or less perplexity on encyclopedia data without using data from the encyclopedia as training data. (Hint : Try to combine a model with news data and a model with article data together.)

In [ ]:
train_1 = sentences_data(['article','news'])
encyclo_data = sentences_data(['encyclopedia'])

In [ ]:
# Fill code here
# 428.85251789073953 (on combined data)
combined_model = getBigramWithInterpolation(train_1)
# print(perplexity(train_1,combined_model))

print('Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data', perplexity(encyclo_data, combined_model))

Perplexity of combine Bigram model with interpolation smoothing on encyclopedia test data 427.50913889506825


## TODO #8 
## Kneser-ney on "News"

<!-- Reimplement equation 4.33 in SLP textbook (https://lagunita.stanford.edu/c4x/Engineering/CS-224N/asset/slp4.pdf) -->

Implement Bigram Knerser-ney LM. The result perplexity should be around 55.06 and 140.76 on train and test data. 


In [ ]:
def getBigramWithKN(data, d = 0.75):
    #Fill code here
    bigram_count = defaultdict(lambda: 0)
    unigram_count = defaultdict(lambda: 0)
    model = defaultdict(lambda: 0)
    word_count = 0
    for sentence in data:
        for (w1, w2) in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          bigram_count[w1, w2] += 1
          unigram_count[w2] += 1
          word_count += 1
        word_count -= 1
        # Word count -1 in each sentences because sentences with 8 word will have 9 bigram-pairs.
    
    front_word_count = dict(Counter(k[0] for k,v in bigram_count.items() if v > 0 ))
    back_word_count = dict(Counter(k[1] for k,v in bigram_count.items() if v > 0))

    c = len([v for k, v in bigram_count.items() if v > 0])

    for k,v in bigram_count.items(): 
      # count same front word that bigrams > 0
      cw = front_word_count[k[0]]

      # count same back word that bigrams > 0
      cwi_1 = back_word_count[k[1]]

      model[k[0], k[1]] = (max((bigram_count[k[0], k[1]] - d), 0) / unigram_count[k[0]]) + \
                          ((d * cw / unigram_count[k[0]]) * (cwi_1/c))

    return model

In [ ]:
# replicate code from Aj. Peerapon

def getBigramWithKN_rep_Aj(data, d = 0.75):
    #Fill code here
    bigram_count = defaultdict(lambda: 0)
    unigram_count = defaultdict(lambda: 0)
    bigram_cn_count = defaultdict(lambda: 0)
    unigram_cn_count = defaultdict(lambda: 0)
    model = defaultdict(lambda: 0)
    # word_count = 0
    for sentence in data:
        for (w1, w2) in bigrams(sentence.split('|'), pad_right=True, pad_left=True):
          bigram_count[w1, w2] += 1
          unigram_count[w2] += 1

    # Create unigrams # bigram counting table
    # 1. counting all elements
    
    bigram_cn1_count = defaultdict(lambda: 0)
    bigram_cn2_count = defaultdict(lambda: 0)
    bigram_list = [k for k in bigram_count.keys() if bigram_count[k] != bigram_count.default_factory()]

    for bigram in bigram_list:
      bigram_cn1_count[bigram[0]] += 1
      bigram_cn2_count[bigram[1]] += 1

    # 2 compute p_cn (continuation prob)
    w1_list = [k for k in bigram_cn1_count.keys() if bigram_cn1_count[k] != bigram_count.default_factory()]
    p_cn = defaultdict(lambda: 0)

    for word in w1_list:
      p_cn[word] = bigram_cn1_count[word]/float(len(bigram_list))
    
    # 3 compute p_kn (knerser-ney prob)
    for x in bigram_list:
      norm_const = (d/unigram_count[x[0]]) * bigram_cn2_count[x[1]]
      model[x] = (max(bigram_count[x]-d,0)/unigram_count[x[0]] + norm_const*p_cn[x[0]])

    return model

In [ ]:
# Fill codehere
model = getBigramWithKN(train)

print(perplexity(train,model))        
print(perplexity(test,model))

# 71.14054002208687
# 174.02464248000433

71.14054002208687
155.09274968738495


In [ ]:
# replicate code from Aj. Peerapon
# I sill got the same perplexity for both train and test sets.

model = getBigramWithKN_rep_Aj(train)

print(perplexity(train,model))        
print(perplexity(test,model))

71.14054002208687
155.09274968738495


## TODO #9
## Neural LM (optional)
do it on news corpus that we splitted into train and test sets at the beginning of this exercise. 

In [ ]:
#there are many ways to do this. e.g.:
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/